In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from pandas.plotting import scatter_matrix

from matplotlib import pyplot as plt
plt.style.use("fivethirtyeight")

pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

In [2]:
# astros2016 = pd.read_csv("data/HOU2016.csv")
astros2017 = pd.read_csv("data/HOU2017.csv")
# astros2018 = pd.read_csv("data/HOU2018.csv")
astros2019 = pd.read_csv("data/HOU2019.csv")
yankees2017 = pd.read_csv("data/NYY2017.csv")
dodgers2017 = pd.read_csv("data/LAD2017.csv")
redsox2017 = pd.read_csv("data/BOS2017.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'data/HOU2017.csv'

In [ ]:
def boolhome(df):
    df.dropna(axis="index", how="all", inplace=True, subset=['Gtm'])
    df.rename(columns={"Unnamed: 3":"Home"}, inplace=True)
    df.replace({"Home":{"@": False, np.nan: True}}, inplace=True)
    df.drop("Unnamed: 0", axis=1, inplace=True)
#     df['GmeSc'] = df['Opp. Starter (GmeSc)'].apply(lambda x: x[-3:-1])
    df['GmeSc'] = df['Opp. Starter (GmeSc)'].str.extract('(\d+)')
    df['GmeSc'] = df['GmeSc'].astype(int)
    df['SoAb'] = df['SO'] / df["AB"]
    df['BbAb'] = df['BB'] / df["AB"]
    df.reset_index(drop=True, inplace=True)
#     df['bbsum'] = df["BB"] + df['SO']
    #df['GmeSc'].apply(lambda x: x.astype(int))
    return df

In [ ]:
astros2017df = boolhome(astros2017)
# astros2018df = boolhome(astros2018)
astros2019df = boolhome(astros2019)
# astros2016df = boolhome(astros2016)
yankees2017df = boolhome(yankees2017)
dodgers2017df = boolhome(dodgers2017)
redsox2017df = boolhome(redsox2017)

In [ ]:
columns = astros2017df.columns

In [ ]:
columns

# GROUPBY home/away

In [ ]:
#Home vs. Away groupby for top teams 
astros_home_gb = astros2017df.groupby("Home")
# yankees_home_gb = yankees2017df.groupby("Home")
# dodgers_home_gb = dodgers2017df.groupby("Home")
# redsox_home_gb = redsox2017df.groupby("Home")

In [ ]:
#Home vs. Away groupby for Astros 2017 and 2019
astros2017_home_gb = astros2017df.groupby("Home")
astros2019_home_gb = astros2019df.groupby("Home")


* ## OBP advantage

In [ ]:
astros_home_obp = astros_home_gb.get_group(True).OBP
astros_away_obp = astros_home_gb.get_group(False).OBP

In [ ]:
astros2017_home_obp = astros2017_home_gb.get_group(True).OBP
astros2017_away_obp = astros2017_home_gb.get_group(False).OBP
astros2019_home_obp = astros2019_home_gb.get_group(True).OBP
astros2019_home_obp = astros2019_home_gb.get_group(False).OBP

In [ ]:
# yankees_home_obp = yankees_home_gb.get_group(True).OBP
# yankees_away_obp = yankees_home_gb.get_group(False).OBP

In [ ]:
# dodgers_home_obp = dodgers_home_gb.get_group(True).OBP
# dodgers_away_obp = dodgers_home_gb.get_group(False).OBP

In [ ]:
# redsox_home_obp = redsox_home_gb.get_group(True).OBP
# redsox_away_obp = redsox_home_gb.get_group(False).OBP

In [ ]:
# # heights: matt vs nick
# fig, ax = plt.subplots(1, figsize=(16, 3))

# ax.scatter(astros_home_obp, 
#            np.repeat(0, len(astros_home_obp)) + \
#            np.random.normal(0, 0.1, len(astros_home_obp)), ## jitter
#            s=45)
# ax.scatter(astros_away_obp, 
#            np.repeat(1, len(astros_away_obp)) + \
#            np.random.normal(0, 0.1, len(astros_home_obp)), 
#            s=45)
# ax.set_yticks([0, 1])
# ax.set_yticklabels(["Home", "Away"]);

#### Write down the Null and Alternative Hypothesis 

**Null:** There is no difference in the OBP between HOME and AWAY games.

**Alternative:** There is a difference in the OBP between HOME and AWAY games.

In [ ]:
def welch_test_statistic(sample_1, sample_2):
    numerator = np.mean(sample_1) - np.mean(sample_2)
    denominator_sq = (np.var(sample_1) / len(sample_1)) + \
                        (np.var(sample_2) / len(sample_2))
    return numerator / np.sqrt(denominator_sq)

In [ ]:
test_statistic = welch_test_statistic(astros_home_obp, astros_away_obp)
print("Welch Test Statistic: {:2.2f}".format(test_statistic))

In [ ]:
def welch_satterhwaithe_df(sample_1, sample_2):
    ss1 = len(sample_1)
    ss2 = len(sample_2)
    df = (
        ((np.var(sample_1)/ss1 + np.var(sample_2)/ss2)**(2.0)) / 
        ((np.var(sample_1)/ss1)**(2.0)/(ss1 - 1) + (np.var(sample_2)/ss2)**(2.0)/(ss2 - 1))
    )
    return df

In [ ]:
df = welch_satterhwaithe_df(astros_home_obp, astros_away_obp)
print("Degrees of Freedom for Welch's Test: {:2.2f}".format(df))

In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
students = stats.t(df)
ax.plot(x, students.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(df))
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis");

**Also, the above is not always true!**  The $T$ statistic only has a t-distribution **under the assumption that the population distributions are Normal**!  

# is this why my plots look weighted towards one side? is my sample normally distributed? Is OBP NOT normally distributed because it's a rolling average?

If the population is very non-normal, the properties of the t-test **will fail**.  You must have some legitimate a-priori reason to believe the populations are approximately normal to use a t-test!


#### Set Significance level

Given the the love I have for my childhood baseball tea (and in a sense my childhood) that's at stake here, and how much they value to friendship, I decide to be reasonably skeptical, so take $\alpha = 0.05$.

#### Calculate p-value


In [ ]:
# p-val
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
students = stats.t(df)
ax.plot(x, students.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(df))
_ = ax.fill_between(x, students.pdf(x), where=(x >= -test_statistic), color="red", alpha=0.25)
_ = ax.fill_between(x, students.pdf(x), where=(x <= test_statistic), color="red", alpha=0.25)
ax.legend()
ax.set_title("p-value Reigon");

In [ ]:
# p_value = students.cdf(test_statistic) + (1 - students.cdf(-test_statistic))
# print("p-value for different average kickflip height: {:2.2f}".format(p_value))

# 0.01 < 0.05 --> there is a statistical difference between HOME and AWAY games.

In [ ]:
# test_statistic = welch_test_statistic(astros_home_obp, astros_away_obp)

# p_value = 1 - students.cdf(test_statistic)
# print("p-value for the OBP for HOME games greater than AWAY games: {:2.3f}".format(p_value))

In [ ]:
astros__obp_ttest = stats.ttest_ind(astros_home_obp, astros_away_obp)

In [ ]:
astros__SoAb_home_ttest = stats.ttest_ind(astros2017_home_obp, astros_away_obp) 

In [ ]:
#astros__SoAb_away_ttest = stats.ttest_ind(astros_home_obp, astros_away_obp)

In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis that 
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros__obp_ttest[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros__obp_ttest[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis")

print(f"Astro's t-statistic is {astros__obp_ttest[0]}, and p-value is {astros__obp_ttest[1]}")

In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
_ = ax.fill_between(x, curve.pdf(x), where=(x >= -astros__obp_ttest[0]), color="red", alpha=0.25)
_ = ax.fill_between(x, curve.pdf(x), where=(x <= astros__obp_ttest[0]), color="red", alpha=0.25)
_ = ax.fill_between(x, curve.pdf(x), where=(x >= .95), color="blue", alpha=0.05)
_ = ax.fill_between(x, curve.pdf(x), where=(x <= -.95), color="blue", alpha=0.05)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis")

print(f"Astro's t-statistic is {astros__obp_ttest[0]}, and p-value is {astros__obp_ttest[1]}")

In [ ]:
#OBP for 2017 season
print(astros2017df.OBP.median())
print(yankees2017df.OBP.median())
print(dodgers2017df.OBP.median())
print(redsox2017df.OBP.median())

In [ ]:
#OBP for 2017 home games
print(astros_home_obp.mean())
# print(yankees_home_obp.mean())
# print(dodgers_home_obp.mean())
# print(redsox_home_obp.mean())

In [ ]:
#OBP for 2017 away games
print(astros_away_obp.mean())
# print(yankees_away_obp.mean())
# print(dodgers_away_obp.mean())
# print(redsox_away_obp.mean())

In [ ]:
#home.get_group(True).OBP.plot.line()

In [ ]:
astros2017df.OBP.plot.line();

In [ ]:
astros_away_obp.plot.line(label="away")
astros_home_obp.plot.line(label="home")
plt.legend()
print(f'home:{astros_home_obp.mean()}')
print(f'away:{astros_away_obp.mean()}')


In [ ]:
astros_away_obp.plot.hist(label="away")
astros_home_obp.plot.hist(label="home")

In [ ]:
# yankees_away_obp.plot.line(label="away")
# yankees_home_obp.plot.line(label="home")
# plt.legend()
# print(f'home:{yankees_home_obp.mean()}')
# print(f'away:{yankees_away_obp.mean()}');

In [ ]:
# dodgers_away_obp.plot.line(label="away")
# dodgers_home_obp.plot.line(label="home")
# plt.legend()
# print(f'home:{dodgers_home_obp.mean()}')
# print(f'away:{dodgers_away_obp.mean()}');

In [ ]:
# redsox_away_obp.plot.line(label="away")
# redsox_home_obp.plot.line(label="home")
# plt.legend()
# print(f'home:{redsox_home_obp.mean()}')
# print(f'away:{redsox_away_obp.mean()}');

#### Is On Base Percentage (OBP) a rolling/weighted average? Can I parce HOME vs AWAY games? Should I consider a different stat like Strikeouts (SO)?

* ## SO advantage
fewer stikeouts/game
expect: astros to have fewer strikeouts than the field

In [ ]:
# astros_home_so = astros_home_gb.get_group(True).SO
# astros_away_so = astros_home_gb.get_group(False).SO

In [ ]:
# yankees_home_so = yankees_home_gb.get_group(True).SO
# yankees_away_so = yankees_home_gb.get_group(False).SO

In [ ]:
# dodgers_home_so = dodgers_home_gb.get_group(True).SO
# dodgers_away_so = dodgers_home_gb.get_group(False).SO

In [ ]:
# redsox_home_so = redsox_home_gb.get_group(True).SO
# redsox_away_so = redsox_home_gb.get_group(False).SO

In [ ]:
# astros_home_so_desc = astros_home_so.describe()

In [ ]:
#astros_home_so_desc

In [ ]:
# print(astros_home_so.mean())
# print(yankees_home_so.mean())
# print(dodgers_home_so.mean())
# print(redsox_home_so.mean())

# list_field_home_so = [yankees_home_so.mean(), dodgers_home_so.mean(), redsox_home_so.mean()]
# arr_field_home_so = np.array(list_field_home_so)

# print((astros_home_so.mean() - arr_field_home_so.mean()))

# arr_field_home_so.mean()


In [ ]:
# print(astros_away_so.mean())
# print(yankees_away_so.mean())
# print(dodgers_away_so.mean())
# print(redsox_away_so.mean())

# list_field_away_so = [yankees_away_so.mean(), dodgers_away_so.mean(), redsox_away_so.mean()]
# arr_field_away_so = np.array(list_field_away_so)

# print((astros_home_so.mean() - arr_field_away_so.mean()))

# arr_field_away_so.mean()

some plot attempts

In [ ]:
# plt.hist(astros_home_so)
# plt.hist(yankees_home_so)
# plt.hist(dodgers_home_so)
# plt.hist(redsox_home_so)
#print(abs(astros_home_so.mean() - yankees_home_so.mean()))
#plt.hist([astros_home_so.mean(), yankees_home_so.mean(), dodgers_home_so.mean(), redsox_home_so.mean()])

In [ ]:
# astros_home_so.plot.bar()
# yankees_home_so.plot.bar()
# abs(astros_home_so - yankees_home_so).plot.bar()

* ## SS/AB
new stat: strikeouts/at bats
expect to be lower than the field

In [ ]:
astros_home_SoAb = astros_home_gb.get_group(True).SoAb
astros_away_SoAb = astros_home_gb.get_group(False).SoAb

In [ ]:
astros2017_home_SoAb = astros2017_home_gb.get_group(True).SoAb
astros2017_away_SoAb = astros2017_home_gb.get_group(False).SoAb
astros2019_home_SoAb = astros2019_home_gb.get_group(True).SoAb
astros2019_home_SoAb = astros2019_home_gb.get_group(False).SoAb

In [ ]:
print(astros_home_SoAb.mean())
print(astros_away_SoAb.mean())

In [ ]:
# yankees_home_SoAb = yankees_home_gb.get_group(True).SoAb
# yankees_away_SoAb = yankees_home_gb.get_group(False).SoAb

In [ ]:
# dodgers_home_SoAb = dodgers_home_gb.get_group(True).SoAb
# dodgers_away_SoAb = dodgers_home_gb.get_group(False).SoAb

In [ ]:
# redsox_home_SoAb = redsox_home_gb.get_group(True).SoAb
# redsox_away_SoAb = redsox_home_gb.get_group(False).SoAb

In [ ]:
# # SS/AB HOME vs. AWAY
# fig, ax = plt.subplots(1, figsize=(16, 3))

# ax.scatter(astros2017df.SoAb, bins = 5,
# #            np.repeat(0, len(astros_home_SoAb)) + \
# #            np.random.normal(0, 0.1, len(astros_home_SoAb)), ## jitter
#            s=45)

# .set_yticks([0, 1])
# ax.set_yticklabels(["Home", "Away"]);

In [ ]:
# SS/AB HOME vs. AWAY
fig, ax = plt.subplots(1, figsize=(, 3))

ax.hist(astros_home_SoAb, bins=8, label="Home", color='#fc4f30', alpha=.5)
ax.hist(astros_away_SoAb, bins=8, label="Away", color='#008fd5', alpha=.2)

plt.ylabel("Frequency")
plt.xlabel("SO/AB")
plt.title("SoAb Home vs. Away for 2018 Regular Season")

ax.legend();

In [ ]:
# SS/AB HOME 2017 vs. 2019
fig, ax = plt.subplots(1, figsize=(16, 3))

ax.hist(astros2017_home_SoAb, bins=8, label="2017", color='#fc4f30', alpha=.5)
ax.hist(astros2019_home_SoAb, bins=8, label="2019", color='#008fd5', alpha=.2)

plt.ylabel("Frequency")
plt.xlabel("SO/AB")

ax.legend();

In [ ]:
# # SS/AB HOME vs. AWAY
# fig, ax = plt.subplots(1, figsize=(16, 3))

# ax.hist(yankees_home_SoAb, bins=20, label="Home", color="blue", alpha=.5)
# ax.hist(yankees_away_SoAb, bins=20, label="Away", color="black", alpha=.2)

# plt.ylabel("Games")
# plt.xlabel("SO/AB")

# ax.legend();

In [ ]:
# # SS/AB HOME vs. AWAY
# fig, ax = plt.subplots(1, figsize=(16, 3))

# ax.hist(dodgers_home_SoAb, bins=20, label="Home", color="purple", alpha=.5)
# ax.hist(dodgers_away_SoAb, bins=20, label="Away", color="black", alpha=.2)

# plt.ylabel("Games")
# plt.xlabel("SO/AB")

# ax.legend();

In [ ]:
# # SS/AB HOME vs. AWAY
# fig, ax = plt.subplots(1, figsize=(16, 3))

# ax.hist(redsox_home_SoAb, bins=20, label="Home", color="red",alpha=.5)
# ax.hist(redsox_away_SoAb, bins=20, label="Away", color="black", alpha=.2)

# plt.ylabel("Games")
# plt.xlabel("SO/AB")

# ax.legend();

In [ ]:
# SS/AB HOME vs. AWAY
# fig, ax = plt.subplots(1,1)

# ax.hist(astros2017df.SoAb)
# ax.hist(astros_home_SoAb, bins=90)
# ax.hist(astros_away_SoAb, alpha=.5, bins=90)
# ax.hist(astros2017df.Date)
# ax.legend(["Home", "Away"]);

#### Write down the Null and Alternative Hypothesis 

**Null:** There is no difference in the SS/AB between HOME and AWAY games.

**Alternative:** There is a difference in the SS/AB between HOME and AWAY games.

In [ ]:
# astros_test_statistic = welch_test_statistic(astros_home_SoAb, astros_away_SoAb)
# print("Welch Test Statistic: {:2.2f}".format(astros_test_statistic))
# yankees_test_statistic = welch_test_statistic(yankees_home_SoAb, yankees_away_SoAb)
# print("Welch Test Statistic: {:2.2f}".format(yankees_test_statistic))
# dodgers_test_statistic = welch_test_statistic(dodgers_home_SoAb, dodgers_away_SoAb)
# print("Welch Test Statistic: {:2.2f}".format(dodgers_test_statistic))
# redsox_test_statistic = welch_test_statistic(redsox_home_SoAb, redsox_away_SoAb)
# print("Welch Test Statistic: {:2.2f}".format(redsox_test_statistic))

In [ ]:
# astros_df = welch_satterhwaithe_df(astros_home_SoAb, astros_away_SoAb)
# print("Degrees of Freedom for Welch's Test: {:2.2f}".format(astros_df))
# yankees_df = welch_satterhwaithe_df(yankees_home_SoAb, yankees_away_SoAb)
# print("Degrees of Freedom for Welch's Test: {:2.2f}".format(yankees_df))
# dodgers_df = welch_satterhwaithe_df(dodgers_home_SoAb, dodgers_away_SoAb)
# print("Degrees of Freedom for Welch's Test: {:2.2f}".format(dodgers_df))
# redsox_df = welch_satterhwaithe_df(redsox_home_SoAb, redsox_away_SoAb)
# print("Degrees of Freedom for Welch's Test: {:2.2f}".format(redsox_df))

#### Write down the Null and Alternative Hypothesis 

**Null:** There is no difference in the SO/AB between HOME and AWAY games.

**Alternative:** There is a difference in the SO/AB between HOME and AWAY games.

# Run the t-test

In [ ]:
astros_home2017away = stats.ttest_ind(astros_home_SoAb, astros_away_SoAb)

In [ ]:
astros_2017home2019 = stats.ttest_ind(astros2017_home_SoAb, astros2019_home_SoAb)

In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros_2017home2019[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros_2017home2019[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home in 2017 vs 2019")

print(f"Astro's t-statistic is {astros_2017home2019[0]}, and p-value is {astros_2017home2019[1]}")


In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home vs Away in 2017
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros_home2017away[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros_home2017away[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home vs Away in 2017")

print(f"Astro's t-statistic is {astros_home2017away[0]}, and p-value is {astros_home2017away[1]}")




In [ ]:
# # p-val
# x = np.linspace(-3, 3, num=250)

# fig, ax = plt.subplots(1, figsize=(16, 3))
# students = stats.t(160) #stats.t_ind()
# ax.plot(x, students.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
# _ = ax.fill_between(x, students.pdf(x), where=(x >= -astros[0]), color="red", alpha=0.25)
# _ = ax.fill_between(x, students.pdf(x), where=(x <= astros[0]), color="red", alpha=0.25)
# ax.legend()
# ax.set_title("p-value Reigon");

In [ ]:
# astros_p_value = students.cdf(astros_test_statistic) + (1 + students.cdf(astros_test_statistic))
# print("p-value for different SS/AB: {:2.2f}".format(astros_p_value))
# yankees_p_value = students.cdf(-yankees_test_statistic) + (1 - students.cdf(yankees_test_statistic))
# print("p-value for different SS/AB: {:2.2f}".format(yankees_p_value))
# dodgers_p_value = students.cdf(-dodgers_test_statistic) + (1 - students.cdf(dodgers_test_statistic))
# print("p-value for different SS/AB: {:2.2f}".format(dodgers_p_value))
# redsox_p_value = students.cdf(-redsox_test_statistic) + (1 - students.cdf(redsox_test_statistic))
# print("p-value for different SS/AB: {:2.2f}".format(redsox_p_value))



In [ ]:
# def welch_satterhwaithe_df(sample_1, sample_2):
#     ss1 = len(sample_1)
#     ss2 = len(sample_2)
#     df = (
#         ((np.var(sample_1)/ss1 + np.var(sample_2)/ss2)**(2.0)) / 
#         ((np.var(sample_1)/ss1)**(2.0)/(ss1 - 1) + (np.var(sample_2)/ss2)**(2.0)/(ss2 - 1))
#     )
#     return df

In [ ]:
# welch_satterhwaithe_df(astros_home_SoAb, astros_away_SoAb)

In [ ]:
# test_statistic = welch_test_statistic(astros_home_SoAb, astros_away_SoAb)

# p_value = 1 - students.cdf(test_statistic)
# print("p-value for the OBP for HOME games greater than AWAY games: {:2.3f}".format(p_value))

In [ ]:
# print(astros_home_SoAb.mean())
# print(yankees_home_SoAb.mean())
# print(dodgers_home_SoAb.mean())
# print(redsox_home_SoAb.mean())

# list_field_home_soab = [yankees_home_SoAb.mean(), dodgers_home_SoAb.mean(), redsox_home_SoAb.mean()]
# arr_field_home_soab = np.array(list_field_home_soab)

# print((astros_home_SoAb.mean() - arr_field_home_soab.mean()))

# arr_field_home_soab.mean()

In [ ]:
# print(astros_away_SoAb.mean())
# print(yankees_away_SoAb.mean())
# print(dodgers_away_SoAb.mean())
# print(redsox_away_SoAb.mean())

# list_field_away_soab = [yankees_away_SoAb.mean(), dodgers_away_SoAb.mean(), redsox_away_SoAb.mean()]
# arr_field_away_soab = np.array(list_field_away_soab)

# print((astros_away_SoAb.mean() - arr_field_away_soab.mean()))

# arr_field_home_soab.mean()

* ## BB/AB
EXPECT: astros to be higher than the field

In [ ]:
astros_home_BbAb = astros_home_gb.get_group(True).BbAb
astros_away_BbAb = astros_home_gb.get_group(False).BbAb

In [ ]:
astros2017_home_BbAb = astros2017_home_gb.get_group(True).BbAb
#astros2017_away_BbAb = astros2017_home_gb.get_group(False).BbAb
astros2019_home_BbAb = astros2019_home_gb.get_group(True).BbAb
#astros2019_home_BbAb = astros2019_home_gb.get_group(False).BbAb

In [ ]:
# yankees_home_BbAb = yankees_home_gb.get_group(True).BbAb
# yankees_away_BbAb = yankees_home_gb.get_group(False).BbAb

In [ ]:
# dodgers_home_BbAb = dodgers_home_gb.get_group(True).BbAb
# dodgers_away_BbAb = dodgers_home_gb.get_group(False).BbAb

In [ ]:
# redsox_home_BbAb = redsox_home_gb.get_group(True).BbAb
# redsox_away_BbAb = redsox_home_gb.get_group(False).BbAb

# how do get these colors but "cheating years" are the same red shade

In [ ]:
# Bb/AB HOME vs. AWAY

fig, ax = plt.subplots(1, figsize=(16, 3))

ax.hist(astros_home_BbAb, bins=8, color='#fc4f30', label="Home", alpha=.4)
ax.hist(astros_away_BbAb, bins=8, color='#008fd5', label="Away", alpha=.4)


plt.ylabel("Games")
plt.xlabel("BB/AB")

ax.legend();


In [ ]:
# Bb/AB HOME 2017 vs 2019

fig, ax = plt.subplots(1, figsize=(16, 3))

ax.hist(astros2017_home_BbAb, bins=8, color='#fc4f30', label="2017 Home", alpha=.4)
ax.hist(astros2019_home_BbAb, bins=8, color='#008fd5', label="2019 Home", alpha=.4)

plt.ylabel("Games")
plt.xlabel("BB/AB")

ax.legend();


In [ ]:
astros_BbAb_home2017away = stats.ttest_ind(astros_home_BbAb, astros_away_BbAb)

In [ ]:
astros__BbAb_2017home2019 = stats.ttest_ind(astros2017_home_BbAb, astros2019_home_BbAb)

In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis 
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros_BbAb_home2017away[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros_BbAb_home2017away[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home in 2017 vs 2019")

print(f"Astro's t-statistic is {astros_BbAb_home2017away[0]}, and p-value is {astros_BbAb_home2017away[1]}")


In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis 
x = np.linspace(-5, 5, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros__BbAb_2017home2019[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros__BbAb_2017home2019[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home in 2017 vs 2019")

print(f"Astro's t-statistic is {astros__BbAb_2017home2019[0]}, and p-value is {astros__BbAb_2017home2019[1]}");


In [ ]:
astros2017_home_BbAb.describe()

In [ ]:
astros2019_home_BbAb.describe()

In [ ]:


# Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home vs Away in 2017
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros_home2017away[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros_home2017away[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home vs Away in 2017")

print(f"Astro's t-statistic is {astros_home2017away[0]}, and p-value is {astros_home2017away[1]}")




In [ ]:
# print(astros_home_BbAb.mean())
# print(yankees_home_BbAb.mean())
# print(dodgers_home_BbAb.mean())
# print(redsox_home_BbAb.mean())

# list_field_home_bbab = [yankees_home_BbAb.mean(), dodgers_home_BbAb.mean(), redsox_home_BbAb.mean()]
# arr_field_home_bbab = np.array(list_field_home_bbab)

# print((astros_home_SoAb.mean() - arr_field_home_soab.mean()))

# arr_field_home_bbab.mean()

In [ ]:
# print(astros_away_BbAb.mean())
# print(yankees_away_BbAb.mean())
# print(dodgers_away_BbAb.mean())
# print(redsox_away_BbAb.mean())

# list_field_away_bbab = [yankees_away_BbAb.mean(), dodgers_away_BbAb.mean(), redsox_away_BbAb.mean()]
# arr_field_away_bbab = np.array(list_field_away_bbab)

# print((astros_away_BbAb.mean() - arr_field_away_bbab.mean()))

# arr_field_away_bbab.mean()

# GAME SCORE

In [ ]:
#astros2017df['GmeSc'].unique()

In [ ]:
#astros2017df['GmeSc'].str.extract('(\d+)')

In [ ]:
#x.apply.astype(int) for x in astros2017df['GmeSc']

In [ ]:
astros_home_GmeSc = astros_home_gb.get_group(True).GmeSc
astros_away_GmeSc = astros_home_gb.get_group(False).GmeSc

In [ ]:
astros2017_home_GmeSc = astros2017_home_gb.get_group(True).GmeSc
#astros2017_away_GmeSc = astros2017_home_gb.get_group(False).GmeSc
astros2019_home_GmeSc = astros2019_home_gb.get_group(True).GmeSc
#astros2019_home_GmeSc = astros2019_home_gb.get_group(False).GmeSc


In [ ]:
# Game Score HOME vs. AWAY

fig, ax = plt.subplots(1, figsize=(16, 3))

ax.hist(astros_away_GmeSc, bins=8, label="Away", alpha=.5, color='#fc4f30')
ax.hist(astros_home_GmeSc, bins=8, label="Home", alpha=.5, color='#008fd5')

plt.ylabel("Frequency")
plt.xlabel("Starting Pitcher Game Score")

ax.legend();
'#fc4f30'

In [ ]:
# SS/AB HOME 2017 vs 2019

fig, ax = plt.subplots(1, figsize=(16, 3))

ax.hist(astros2017_home_GmeSc, bins=8, color='#fc4f30', label="2017 Home", alpha=.5)
ax.hist(astros2019_home_GmeSc, bins=8, color='#008fd5', label="2019 Home", alpha=.5)

plt.ylabel("Frequency")
plt.xlabel("Starting Pitcher Game Score")
plt.title("test")
ax.legend();


In [ ]:
astros_away_GmeSc.describe()

In [ ]:
astros_home_GmeSc.describe()

In [ ]:
astros2017_home_GmeSc.describe()

In [ ]:
astros2019_home_GmeSc.describe()

In [ ]:
astros_GmeSc_home2017away = stats.ttest_ind(astros_home_GmeSc, astros_away_GmeSc)


In [ ]:
astros__GmeSc_2017home2019 = stats.ttest_ind(astros2017_home_GmeSc, astros2019_home_GmeSc)
    

In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis 
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros_GmeSc_home2017away[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros_GmeSc_home2017away[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home in 2017 vs 2019")

print(f"Astro's t-statistic is {astros_GmeSc_home2017away[0]}, and p-value is {astros_GmeSc_home2017away[1]}");



In [ ]:
# Distribution of Welsh's Test Statistic Under the Null Hypothesis 
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros__GmeSc_2017home2019[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros__GmeSc_2017home2019[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home in 2017 vs 2019")

print(f"Astro's t-statistic is {astros__GmeSc_2017home2019[0]}, and p-value is {astros__GmeSc_2017home2019[1]}")


In [ ]:


# Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home vs Away in 2017
x = np.linspace(-3, 3, num=250)

fig, ax = plt.subplots(1, figsize=(16, 3))
curve = stats.t(160)
ax.plot(x, curve.pdf(x), linewidth=2, label="Degree of Freedom: {:2.2f}".format(160))
ax.fill_between(x, curve.pdf(x), where=(x >= -astros_home2017away[0]), color="red", alpha=0.25)
ax.fill_between(x, curve.pdf(x), where=(x <= astros_home2017away[0]), color="red", alpha=0.25)
ax.legend()
ax.set_title("Distribution of Welsh's Test Statistic Under the Null Hypothesis for Home vs Away in 2017")

print(f"Astro's t-statistic is {astros_home2017away[0]}, and p-value is {astros_home2017away[1]}")


